In [ ]:
import pandas as pd

file_path = 'data/hcm_weather.csv'
df = pd.read_csv(file_path)

In [67]:
# Số dòng/cột
print("Số dòng:", df.shape[0])
print("Số dòng:", df.shape[1])

Số dòng: 128448
Số dòng: 11


In [68]:
# xem qua dữ liệu
df.head(10)

,latitude,longitude,datetime,temperature (C degree),humidity (%),precipitation (mm),windspeed (km/h),winddirection (degrees),cloudcover (%),pressure (hPa),dewpoint (C degree)
0,10.775388,106.702825,2024-01-01T00:00,25.6,88,0.0,5.7,145,99,1011.5,23.4
1,10.775388,106.702825,2024-01-01T01:00,25.3,90,0.0,3.6,143,96,1011.2,23.5
2,10.775388,106.702825,2024-01-01T02:00,25.3,88,0.0,0.5,135,96,1010.7,23.2
3,10.775388,106.702825,2024-01-01T03:00,24.9,91,0.0,4.7,9,98,1010.5,23.3
4,10.775388,106.702825,2024-01-01T04:00,24.6,92,0.0,8.0,352,100,1010.4,23.2
5,10.775388,106.702825,2024-01-01T05:00,24.4,92,0.0,7.1,345,99,1010.9,23.0
6,10.775388,106.702825,2024-01-01T06:00,24.2,92,0.0,6.3,347,98,1011.2,22.8
7,10.775388,106.702825,2024-01-01T07:00,24.5,90,0.0,5.2,348,98,1011.8,22.7
8,10.775388,106.702825,2024-01-01T08:00,26.2,78,0.0,7.1,345,99,1013.0,22.1
9,10.775388,106.702825,2024-01-01T09:00,28.1,68,0.0,6.3,336,100,1013.6,21.6


In [69]:
# Các cột và định dạng từng cột:
cols = df.columns
print(df.dtypes)

latitude                   float64
longitude                  float64
datetime                    object
temperature (C degree)     float64
humidity (%)                 int64
precipitation (mm)         float64
windspeed (km/h)           float64
winddirection (degrees)      int64
cloudcover (%)               int64
pressure (hPa)             float64
dewpoint (C degree)        float64
dtype: object


Các cột có kiểu dữ liệu phù hợp, chỉ có cột 'datetime' đang là kiểu object thay vì kiểu datetime. Ngoài ra, nhiều dòng có cùng date, khác time cho nên ta tách date time thành 2 cột thuộc tính để dễ sử dụng khi phân tích

In [70]:
# Định dạng datetime từ kiểu object thành kiểu datetime
df['datetime'] = pd.to_datetime(df['datetime'])

# Tách thêm 2 cột date và time
df['date'] = pd.to_datetime(df['datetime'].dt.date, format='%d/%m/%Y')
df['time'] = pd.to_datetime(df['datetime'].dt.date, format='%H:%M:%S')
df = df.drop(columns=['datetime']) # drop cột 'datetime' sau khi tách

# Xem các cột sau khi định dạng và tách
print(df['date'].head())
print(df['time'].head())

0   2024-01-01
1   2024-01-01
2   2024-01-01
3   2024-01-01
4   2024-01-01
Name: date, dtype: datetime64[ns]
0   2024-01-01
1   2024-01-01
2   2024-01-01
3   2024-01-01
4   2024-01-01
Name: time, dtype: datetime64[ns]


In [71]:
# Số dòng thiếu ở mỗi cột:
print(df.isnull().sum())

latitude                   0
longitude                  0
temperature (C degree)     0
humidity (%)               0
precipitation (mm)         0
windspeed (km/h)           0
winddirection (degrees)    0
cloudcover (%)             0
pressure (hPa)             0
dewpoint (C degree)        0
date                       0
time                       0
dtype: int64


Cột 'dewpoint' có toàn bộ dữ liệu rỗng --> xoá cột này

In [73]:
# xoá cột 'dewpoint'
df = df.drop(columns=['dewpoint (C degree)'])

In [74]:
# Số dòng trùng lặp
duplicate_count = df.duplicated().sum()
print(f"Số dòng trùng lặp: {duplicate_count}")

Số dòng trùng lặp: 0


In [76]:
# Khoảng giá trị của các cột
numeric_cols = df.select_dtypes(include=['number']).columns # chọn các cột dữ liệu dạng số

for col in numeric_cols:
    if df[col].isna().all(): # cột rỗng --> skip
        continue

    min_val = df[col].min()
    max_val = df[col].max()
    print(f"{col:<25} {min_val} - {max_val}")

latitude                  10.41138 - 10.995955
longitude                 106.511271 - 107.136224
temperature (C degree)    18.5 - 38.9
humidity (%)              23 - 100
precipitation (mm)        0.0 - 24.7
windspeed (km/h)          0.0 - 37.6
winddirection (degrees)   1 - 360
cloudcover (%)            0 - 100
pressure (hPa)            999.9 - 1019.2


--- KHOẢNG GIÁ TRỊ THAM CHIẾU ---
- latitude: -90 đến 90
- longitude: -180 đến 180
- temperature: Mức thấp kỷ lục (-89.2°C) đến Mức cao kỷ lục (56.7°C)
- humidity: 0% đến 100%
- precipitation: >= 0 mm
- windspeed: >= 0 km/h
- winddirection: 0° đến 360°
- cloudcover: 0% đến 100%
- pressure: 870 hPa đến 1085 hPa

So sánh với khoảng giá trị tham chiếu, các thuộc tính đều hợp lý, không có điểm bất thường

In [65]:
# xem xét các giá trị lat/long
locations = df[['latitude', 'longitude']].drop_duplicates()

print(f"Số lượng cặp (lat, long): {len(locations)}")
print("\nChi tiết các tọa độ:")
print(locations)

Số lượng cặp (lat, long): 3

Chi tiết các tọa độ:
       latitude  longitude
0       10.5359   106.8825
16056   10.9189   106.4479
32112   10.5782   106.7880


Xem xét 2 cột 'latitude' và 'longtitude', nhóm có những cân nhắc sau:

**Hiện trạng**

Dữ liệu tập trung tại 3 tọa độ cố định đại diện cho các vùng khí hậu khác nhau (TP.HCM, Biên Hòa...).

**Vấn đề**

Cần xác định vai trò của latitude và longitude trong việc dự báo mưa (nên giữ hay bỏ 2 cột này?). Liệu sự khác biệt địa lý có ảnh hưởng đến ngưỡng mưa (micro-climate) không?

**Quyết định**

- Giữ lại Lat/Long: Không gộp để tránh mất thông tin cục bộ

- Khoanh vùng và phân loại lại khu vực (Zoning) thay vì dùng tọa độ thô

- Chia nhóm các khu vực: khu vực trung tâm (Q1, Q3...), khu vực ven biển (Cần Giờ), khu vực lân cận

- Mục tiêu: phân tích được đặc thù mưa của từng vùng địa lý cụ thể thay vì làm bằng phẳng (flatten) hết toàn vùng